In [1]:
import skimage
from skimage import io
from skimage import morphology
import os
import sys
import numpy as np
from matplotlib import pyplot as plt
from math import log
from scipy import misc as mi
import scipy
from scipy import signal as Sig

def negIm(a):
    b=255-a
    b=np.where(b>255,255,b)
    b=np.where(b<0,0,b)
    return b
            
def refK(k):
    k=np.flip(k, axis=0)
    k=np.flip(k,axis=1)
    return k



def fastDial(a,k):
    #m=int(2*log(len(k))+2) #large enough for filter, larger values cause overflow
    a=np.array(a,dtype=np.float)
    k=np.array(k,dtype=np.float)
    m=0.16 #both for k1 and b2
    ma=m*a
    exp_ma=np.exp(ma)
    mk=m*k
    exp_mk=np.exp(mk)
    exp_mc=Sig.convolve(exp_ma, exp_mk, mode='same',method='fft') #probem lies in this step if fft used
    mc=np.log(exp_mc)
    c=mc/m
    c=np.where(c==np.nan,128,c)
    c=np.where(c>255,255,c)
    c=np.where(c<0,0,c)
    c=np.array(c,dtype=np.uint8)
    return c

def fastErod(a,k):
    #uses duality principle
    return  negIm(fastDial(negIm(a),refK(k)))

def fastOp(a,k):
    b=fastErod(a,k)
    b=fastDial(b,k)
    return b

def fastCl(a,k):
    b=fastDial(a,k)
    b=fastErod(b,k)
    return b

In [2]:
#filters: ref: ref: An approach to color-morphology based on Einstein addition and Loewner order
smSq=[[0,0,0],[0,0,0],[0,0,0]]
square=[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
cross=[[0,-256,-256,-256,0],[-256,0,-256,0,-256],[-256,-256,0,-256,-256] ,[-256,0,-256,0,-256],[0,-256,-256,-256,0]]
diamond=[]
for i in range(21):
    b=[]
    for j in range(21):
        b.append(-256)
        if(i==10):
            b[j]=0
        if((i<10)and(j<=(10+i))and(j>=(10-i))):
            b[j]=0
        if((i>10)and(j>=(i-10))and(j<=(30-i))):
            b[j]=0
    diamond.append(b)
#print(diamond)

In [3]:
from scipy import ndimage, misc
def footprint(k):
    a=np.where(k>=0,1,0)
    return a
def dial(a,k):
    return ndimage.grey_dilation(a,structure=k,footprint=footprint(k))
def erod(a,k):
    return ndimage.grey_erosion(a,structure=k,footprint=footprint(k))
def op(a,k):
    return ndimage.grey_opening(a,structure=k,footprint=footprint(k))
def cl(a,k):
    return ndimage.grey_closing(a,structure=k,footprint=footprint(k))

In [4]:
#Morphological Gradients (Component Wise)
#ref: An approach to color-morphology based on Einstein addition and Loewner order
def Beuchergradient(img, k):
    a=dial(img,k)
    b=erod(img,k)
    img_bGrad=np.where(a<b,0,a-b)
    return img_bGrad
def fastBeuchergradient(img, k):
    a=fastDial(img,k)
    b=fastErod(img,k)
    img_bGrad=np.where(a<b,0,a-b)
    return img_bGrad
def intGradient(img,k):
    b=erod(img,k)
    img_iGrad=np.where(img<b,0,img-b)
    return img_iGrad
def fastIntGradient(img,k):
    b=fastErod(img,k)
    img_iGrad=np.where(img<b,0,img-b)
    return img_iGrad
def extGradient(img,k):
    a=dial(img,k)
    img_iGrad=np.where(img>a,0,a-img)
    return img_iGrad
def fastExtGradient(img,k):
    a=fastDial(img,k)
    img_iGrad=np.where(img>a,0,a-img)
    return img_iGrad
def topHat(img,k):
    a=op(img,k)
    img_iGrad=np.where(img<a,0,img-a)
    return img_iGrad
def fastTopHat(img,k):
    a=fastOp(img,k)
    img_iGrad=np.where(img<a,0,img-a)
    return img_iGrad
def blackTopHat(img,k):
    a=cl(img,k)
    img_iGrad=np.where(img>a,0,a-img)
    return img_iGrad
def fastBlackTopHat(img,k):
    a=fastCl(img,k)
    img_iGrad=np.where(img>a,0,a-img)
    return img_iGrad 
#def morphLaplacian(img,k): 
#    a=extGradient(img,k)
#    b=intGradient(img,k)
#    img_Lap=np.subtract(a,b)    #discuss negative pixel value 